#**Word2Vec實作**


*   字詞所代表的意義非常多元，在不同狀況下，會代表不同意思。要把多元意思用單一向量表示，則必須要進行word embedding的動作，也就是把高維向量降為低維向量的過程
*   之前介紹過，利用分散式表示法來表達字詞向量，例如PMI、SVD..統計法..等
*   2013年神經網路盛行後，Tomas Mikolov利用神經網路訓練方式，來獲得字詞的表達向量，獲得很棒的成果。一般認為是利用神經網路模擬人類的理解能力，獲得不錯的分布空間所得到的成果。
*   本範例以維基百科wiki部分資料作範例
*   資料來源：https://dumps.wikimedia.org/zhwiki/20231201/zhwiki-20231201-pages-articles-multistream1.xml-p1p187712.bz2
*   利用結巴分詞(jieba)進行斷詞，gensim套件進行word2vec計算
*   本範例約需1小時長時間執行






In [1]:
!wget https://dumps.wikimedia.org/zhwiki/20231201/zhwiki-20231201-pages-articles-multistream1.xml-p1p187712.b

--2024-01-04 04:50:11--  https://dumps.wikimedia.org/zhwiki/20231201/zhwiki-20231201-pages-articles-multistream1.xml-p1p187712.b
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.142, 2620:0:861:2:208:80:154:142
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.142|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-01-04 04:50:12 ERROR 404: Not Found.



##opencc是繁簡轉換工具

In [2]:
!pip install opencc-python-reimplemented # 安裝套件

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.8/481.8 kB 5.3 MB/s eta 0:00:00


##gensim是訓練word2vec的函式庫

In [ ]:
from gensim.corpora import WikiCorpus

wiki_corpus = WikiCorpus('zhwiki-20231201-pages-articles-multistream1.xml-p1p187712.bz2',

In [ ]:
wiki_corpus

In [ ]:
next(iter(wiki_corpus.get_texts()))[:10]

##把wiki的資料檔案，轉換成連續文字的txt檔案

In [ ]:
text_num = 0

# 開啟 'wiki_text.txt' 檔案，使用 utf-8 編碼方式寫入
with open('wiki_text.txt', 'w', encoding='utf-8') as f:
    # 迴圈處理從 wiki_corpus 取得的每一篇文章
    for text in wiki_corpus.get_texts():
        # 將文章內容以空格連接，並寫入檔案，每行為一篇文章
        f.write(' '.join(text) + '\n')

        # 文章數量加一
        text_num += 1

        # 每處理 10,000 篇文章印出一條訊息
        if text_num % 10000 == 0:
            print('{} articles processed.'.format(text_num))

    # 最後印出總處理文章數
    print('{} articles processed.'.format(text_num))

In [ ]:
import jieba
from opencc import OpenCC

# 初始化 OpenCC，設定轉換為繁體中文
cc = OpenCC('s2t')

# 讀取 'wiki_text.txt' 檔案中的文本資料
train_data = open('wiki_text.txt', 'r', encoding='utf-8').read()

# 將文本進行簡繁體轉換
train_data = cc.convert(train_data)

# 使用 jieba 對文本進行中文斷詞
train_data = jieba.lcut(train_data)

# 移除空白字詞
train_data = [word for word in train_data if word != '']

# 將斷詞結果以空格連接成字串
train_data = ' '.join(train_data)

# 將處理後的文本保存到 'seg.txt' 檔案中
open('seg.txt', 'w', encoding='utf-8').write(train_data)


In [ ]:
from gensim.models import word2vec

# 設定
seed = 666
sg = 0  # Skip-gram 模型
window_size = 10
min_count = 1
workers = 8
batch_words = 10000

# 從 'seg.txt' 讀取處理後的文本資料
train_data = word2vec.LineSentence('seg.txt')

# 使用 Word2Vec 訓練詞向量模型
model = word2vec.Word2Vec(
    train_data,
    min_count=min_count,
    workers=workers,
    window=window_size,
    sg=sg,
    seed=seed,
    batch_words=batch_words
)

# 儲存訓練好的 Word2Vec 模型
model.save('word2vec.model')

In [ ]:
from gensim.models import word2vec

# 載入先前訓練好的 Word2Vec 模型
model = word2vec.Word2Vec.load('word2vec.model')

# 指定詞語
string = '電腦'
print(string)

# 列印與指定詞語最相似的詞語及相似度
for item in model.wv.most_similar(string):
    print(item)
